<a href="https://colab.research.google.com/github/MaiteFinzi/imersao-alura-gemini/blob/master/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [10]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [11]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [12]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [13]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [58]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(tema, idade):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa que ajudará mamães, papais, avós e avôs, tios e tias, ou qualquer outro responsável, maior de idade,
        a responder as perguntas de crianças na faixa etaria de 03 a 12 anos.
        Você atua como psicólogo pediatra etem didática para conversar e ensinar crianças de 03 a 12 anos de idade.
        A sua tarefa é usar a ferramenta de busca do google (google_search)
        para buscar respostas curtas e que esclareçam as duvidas dessas crianças.
        Você deve levar em consideração que sua resposta deverá ser diferente para os diferentes tipos de idades.
        Sua resposta deve conter palavras fáceis de entender para cada idade.
        Foque em no máximo 3 tipos de resposta para que os responsáveis possam escolher qual seria melhor para responderem a seus filhos.
        Se um tema for sobre sexualidade não utilize palavras de nível técnico, tente contar uma breve história.
        Responda em português. e se tiver alguma pergunta em outro idioma, traduza para o português e dê exemplos.
        Se baseie nas informações mais recentes para que sua resposta seja mais atual.

        """,
        description="Agente que busca respostas no Google, para ajudar os pais e responsáveis de crianças de 3 a 12 anos de idade.",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tema: {tema}\nIdade: {idade}"

    respostas = call_agent(buscador, entrada_do_agente_buscador)
    return respostas


In [59]:
################################################
# --- Agente 2: Planejador de respostas --- #
################################################
def agente_planejador(tema, respostas_buscadas):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de respostas, especialista em responder as perguntas curiosas de crianças de 03 a 12 anos de idade.
        Com base na lista de respostas mais adequadas a cada idade e mais recentes você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar com base nas idades para
        cada uma das crianças.
        Ao final, você irá escolher a resposta mais relevante entre eles com base nas suas pesquisas
        e retornar essa resposta.
        """,
        description="Agente que planeja e responde para ajudar os pais e responsáveis de crianças de 3 a 12 anos de idade.",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tema:{tema}\nRespostas buscadas: {respostas_buscadas}"
    # Executa o agente
    plano_da_resposta = call_agent(planejador, entrada_do_agente_planejador)
    return plano_da_resposta

In [61]:
# Inserir lógica do sistema de agentes ################################################
from datetime import date
import textwrap
from IPython.display import display, Markdown
# Importar a exceção específica do gRPC para melhor tratamento
import grpc


data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Iniciando o Sistema de Criação de Respostas para ajudar pais e responsáveis a responderem as curiosidades de crianças de 03 a 12 anos de idade.")

tema_completo = input("Por favor, digite a idade da criança e o tema sobre o qual você quer ajuda(Ex: 6 anos De onde vêm os bebês): ")

if not tema_completo:
    print("Você esqueceu de digitar a idade e o tema!")
else:
    try:
        # Tenta dividir a entrada considerando a idade no início
        partes = tema_completo.split(maxsplit=1) # Divide em no máximo 2 partes no primeiro espaço
        if len(partes) < 2:
             # Se não houver espaço, tenta separar por "anos"
             partes = tema_completo.split(" anos ", maxsplit=1)
             if len(partes) < 2:
                  # Se ainda não separou, considera a entrada inválida
                  raise ValueError("Formato de entrada inválido.")

        idade_str = partes[0].strip() # Remove espaços em branco antes e depois
        tema = partes[1].strip().replace("'", "") # Remove espaços em branco e aspas simples restantes

        # Tenta converter a idade para um número inteiro para validação básica
        idade = int(idade_str)

        print(f"Maravilha! Vamos então criar uma resposta para uma criança de {idade} anos sobre {tema}.")

        # Chama o agente buscador com o tópico e a data de hoje
        print("Chamando Agente 1 (Buscador)...")
        try:
            # Note: agente_buscador foi definido anteriormente para aceitar tema e idade
            # A entrada original estava usando data_de_hoje, mas o traceback
            # fornecido no início indica que a função original aceitava tema e idade.
            # Vamos manter a chamada consistente com a definição original fornecida.
            respostas_buscadas = agente_buscador(tema, idade)
            print("--- 📝 Resultado do Agente 1 (Buscador) ---\n")
            display(to_markdown(respostas_buscadas))
            print("--------------------------------------------------------------")
        except Exception as e:
            print(f"Erro na execução do Agente 1 (Buscador): {e}")
            respostas_buscadas = None # Define como None para evitar erros subsequentes

        # Chama o agente planejador
        if respostas_buscadas: # Procede apenas se o Agente 1 teve sucesso
            print("Chamando Agente 2 (Planejador)...")
            try:
                plano_da_resposta = agente_planejador(tema, respostas_buscadas)
                print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
                display(to_markdown(plano_da_resposta))
                print("--------------------------------------------------------------")
                # Inspecionar a saída do Agente 2
                print(f"Saída do Agente 2 (plano_da_resposta) - Tipo: {type(plano_da_resposta)}")
                print(f"Saída do Agente 2 (plano_da_resposta) - Conteúdo (primeiros 500 chars): {plano_da_resposta[:500] if plano_da_resposta else 'N/A'}")
                print("--------------------------------------------------------------")
            except Exception as e:
                print(f"Erro na execução do Agente 2 (Planejador): {e}")
                plano_da_resposta = None # Define como None para evitar erros subsequentes
        else:
            plano_da_resposta = None  # Define como None se o Agente 1 falhou
            print("Agente 2 (Planejador) não executado devido a falha no Agente 1.")


    except (IndexError, ValueError) as e:
        print(f"Erro de entrada: {e}")
        print("Formato de entrada inválido. Por favor, digite a idade e o tema no formato 'idade anos tema'.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado no fluxo principal: {e}")

# Adiciona este bloco para exibir a resposta final do Agente 2, se ele foi executado com sucesso
if 'plano_da_resposta' in locals() and plano_da_resposta is not None:
    print("\n--- 🚀 Resposta Final (Gerada pelo Agente 2) ---\n")
    display(to_markdown(plano_da_resposta))
    print("--------------------------------------------------------------")
else:
    print("\n--- ❌ Resposta Final não Gerada ---")
    print("A resposta final não pôde ser gerada devido a um erro ou entrada inválida.")
    print("--------------------------------------------------------------")

Iniciando o Sistema de Criação de Respostas para ajudar pais e responsáveis a responderem as curiosidades de crianças de 03 a 12 anos de idade.
Por favor, digite a idade da criança e o tema sobre o qual você quer ajuda(Ex: 6 anos De onde vêm os bebês): 5 anos qual é o maior planeta do sitema solar
Maravilha! Vamos então criar uma resposta para uma criança de 5 anos sobre anos qual é o maior planeta do sitema solar.
Chamando Agente 1 (Buscador)...
--- 📝 Resultado do Agente 1 (Buscador) ---



> Olá! Que pergunta legal! Para responder a essa pergunta, veja o que encontrei:
> 
> 
> É uma pergunta muito interessante! A resposta pode ser dada de algumas formas, dependendo do que você quer saber:
> 
> **Resposta 1:**
> 
> Sabe, no nosso Sistema Solar, temos vários planetas, como a Terra onde vivemos. Mas, Júpiter é o grandão! Ele é tão grande que, se juntássemos todos os outros planetas, ainda seria maior. Júpiter é como um gigante amigável no espaço.
> 
> **Resposta 2:**
> 
> Imagine que os planetas são bolas de diferentes tamanhos. Júpiter seria como uma bola de praia enorme, bem maior que todas as outras bolas juntas. Ele é feito de gases, e tem até listras coloridas e uma grande mancha vermelha, que é como uma tempestade gigante que dura há muito tempo.
> 
> **Resposta 3:**
> 
> Júpiter é um planeta muito especial! Ele é o quinto planeta a partir do Sol e o maior de todos. Ele é tão grande que caberiam mais de 1300 planetas do tamanho da Terra dentro dele. Ele tem muitas luas e alguns anéis, mas não tão bonitos quanto os de Saturno.
> 
> Espero que essas explicações tenham ajudado você a entender melhor qual é o maior planeta do Sistema Solar!
> 


--------------------------------------------------------------
Chamando Agente 2 (Planejador)...

--- 📝 Resultado do Agente 2 (Planejador) ---



> Para responder à pergunta de forma adequada para crianças de diferentes idades (3-12 anos), podemos abordar os seguintes pontos, ajustando a linguagem e a complexidade:
> 
> 1.  **Introdução:** Apresentar o Sistema Solar de forma simples, mencionando que ele é composto por planetas de diferentes tamanhos.
> 
> 2.  **Identificação do maior planeta:** Revelar que Júpiter é o maior planeta do Sistema Solar.
> 
> 3.  **Comparação de tamanho:** Utilizar analogias e comparações para ajudar as crianças a visualizarem o tamanho de Júpiter em relação aos outros planetas e à Terra.
> 
> 4.  **Características de Júpiter:** Descrever algumas características marcantes de Júpiter, como as listras coloridas e a Grande Mancha Vermelha, de forma lúdica e interessante.
> 
> 5.  **Curiosidades:** Compartilhar curiosidades sobre Júpiter, como o número de luas e a presença de anéis, adaptando o nível de detalhe à idade da criança.
> 
> 6.  **Relevância:** Destacar a importância de Júpiter no Sistema Solar e como ele influencia outros corpos celestes.
> 
> 
> Olá! Que pergunta interessante! Você quer saber qual é o maior planeta do Sistema Solar?
> 
> Imagine que o Sistema Solar é como uma vizinhança onde vivem vários planetas, cada um com um tamanho diferente. Júpiter é o grandão dessa turma! Ele é tão grande que, se juntássemos todos os outros planetas, ainda não seria do tamanho dele. Júpiter é como um gigante amigável que vive lá no espaço.
> 
> Júpiter é feito de gases e tem listras coloridas e uma enorme mancha vermelha. Essa mancha é como uma tempestade gigante que está acontecendo lá há muito, muito tempo.
> 
> Para você ter uma ideia, Júpiter é tão grande que mais de 1300 planetas do tamanho da Terra poderiam caber dentro dele! Ele também tem muitas luas, como pequenos satélites naturais, e alguns anéis, mas eles não são tão bonitos e chamativos como os de Saturno.
> 
> Júpiter é muito importante no Sistema Solar. Ele é como um super-herói que protege a Terra de muitos asteroides perigosos. Incrível, né?
> 


--------------------------------------------------------------
Saída do Agente 2 (plano_da_resposta) - Tipo: <class 'str'>
Saída do Agente 2 (plano_da_resposta) - Conteúdo (primeiros 500 chars): Para responder à pergunta de forma adequada para crianças de diferentes idades (3-12 anos), podemos abordar os seguintes pontos, ajustando a linguagem e a complexidade:

1.  **Introdução:** Apresentar o Sistema Solar de forma simples, mencionando que ele é composto por planetas de diferentes tamanhos.

2.  **Identificação do maior planeta:** Revelar que Júpiter é o maior planeta do Sistema Solar.

3.  **Comparação de tamanho:** Utilizar analogias e comparações para ajudar as crianças a visualiza
--------------------------------------------------------------

--- 🚀 Resposta Final (Gerada pelo Agente 2) ---



> Para responder à pergunta de forma adequada para crianças de diferentes idades (3-12 anos), podemos abordar os seguintes pontos, ajustando a linguagem e a complexidade:
> 
> 1.  **Introdução:** Apresentar o Sistema Solar de forma simples, mencionando que ele é composto por planetas de diferentes tamanhos.
> 
> 2.  **Identificação do maior planeta:** Revelar que Júpiter é o maior planeta do Sistema Solar.
> 
> 3.  **Comparação de tamanho:** Utilizar analogias e comparações para ajudar as crianças a visualizarem o tamanho de Júpiter em relação aos outros planetas e à Terra.
> 
> 4.  **Características de Júpiter:** Descrever algumas características marcantes de Júpiter, como as listras coloridas e a Grande Mancha Vermelha, de forma lúdica e interessante.
> 
> 5.  **Curiosidades:** Compartilhar curiosidades sobre Júpiter, como o número de luas e a presença de anéis, adaptando o nível de detalhe à idade da criança.
> 
> 6.  **Relevância:** Destacar a importância de Júpiter no Sistema Solar e como ele influencia outros corpos celestes.
> 
> 
> Olá! Que pergunta interessante! Você quer saber qual é o maior planeta do Sistema Solar?
> 
> Imagine que o Sistema Solar é como uma vizinhança onde vivem vários planetas, cada um com um tamanho diferente. Júpiter é o grandão dessa turma! Ele é tão grande que, se juntássemos todos os outros planetas, ainda não seria do tamanho dele. Júpiter é como um gigante amigável que vive lá no espaço.
> 
> Júpiter é feito de gases e tem listras coloridas e uma enorme mancha vermelha. Essa mancha é como uma tempestade gigante que está acontecendo lá há muito, muito tempo.
> 
> Para você ter uma ideia, Júpiter é tão grande que mais de 1300 planetas do tamanho da Terra poderiam caber dentro dele! Ele também tem muitas luas, como pequenos satélites naturais, e alguns anéis, mas eles não são tão bonitos e chamativos como os de Saturno.
> 
> Júpiter é muito importante no Sistema Solar. Ele é como um super-herói que protege a Terra de muitos asteroides perigosos. Incrível, né?
> 


--------------------------------------------------------------


data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Iniciando o Sistema de Criação de Respostas para ajudar pais e responsáveis a responderem as curiosidades de crianças de 03 a 12 anos de idade.")

# --- Obter o Tópico do Usuário ---
tema_completo = input(" Por favor, digite a idade da criança e o tema sobre o qual você quer ajuda: ")

# Inserir lógica do sistema de agentes ################################################
if not tema_completo:
    print("Você esqueceu de digitar a idade e o tema!")
else:
    try:
        partes = tema_completo.split(" anos '")
        idade = partes[0]
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Iniciando o Sistema de Criação de Respostas para ajudar pais e responsáveis a responderem as curiosidades de crianças de 03 a 12 anos de idade.")

# --- Obter o Tópico do Usuário ---
tema_completo = input(" Por favor, digite a idade da criança e o tema sobre o qual você quer ajuda: ")

# Inserir lógica do sistema de agentes ################################################
if not tema_completo:
    print("Você esqueceu de digitar a idade e o tema!")
else:
    try:
        partes = tema_completo.split(" anos '")
        idade = partes[0]
        tema = partes[1].replace("'", "") # Remove aspas simples restantes
        print(f"Maravilha! Vamos então criar uma resposta para uma criança de {idade} anos sobre {tema}.")
        respostas_buscadas = agente_buscador(tema, data_de_hoje)
        print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
        display(to_markdown(respostas_buscadas))
        print("--------------------------------------------------------------")

        plano_da_resposta = agente_planejador(tema, respostas_buscadas)
        print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
        display(to_markdown(plano_da_resposta))
        print("--------------------------------------------------------------")

        rascunho_da_resposta = agente_redator(tema, plano_da_resposta)
        print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
        display(to_markdown(rascunho_da_resposta))
        print("--------------------------------------------------------------")

        post_final = agente_revisor(tema, rascunho_da_resposta)
        print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
        display(to_markdown(post_final)) # Corrected variable name
        print("--------------------------------------------------------------")
    except IndexError:
        print("Formato de entrada inválido. Por favor, digite no formato 'idade anos tema'.")
    print("--------------------------------------------------------------")

    plano_da_resposta = agente_planejador(tema, respostas_buscadas)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_da_resposta))
    print("--------------------------------------------------------------")

    rascunho_da_resposta = agente_redator(tema, plano_de_resposta)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_da_resposta))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(tema, rascunho_da_resposta)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(resposta_final))
    print("--------------------------------------------------------------")